## Purpose: Try different models-- Part3.
### Grid search.

In [39]:
# import dependencies.
import pandas as pd
import numpy as np

from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, r2_score, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC

##  Grid search with upsampling, with scaling.

#### STEP1: Read in dataset.  Remove data from 2016-2019.
- data from 2016-2018 will be used to bs test the model.
- data from 2019 will be used to predict the winners of the 2019 WS.

In [40]:
# read in the data.
team_data = pd.read_csv("../Resources/clean_data.csv")
del team_data["Unnamed: 0"]
team_data.head()

,team,year,A,DP,E,GS2,INN,PB,PO,TC,...,OBP1,R1,SHO,SO1,SV,TBF,W,WHIP,WP,winners
0,St. Louis Cardinals,2019,1033,114,43,936,8313.0,3,2771,3847,...,0.319,456,4,895,33,3896,56,1.29,21,0
1,Arizona Diamondbacks,2019,1010,83,45,945,8538.0,2,2846,3901,...,0.315,472,7,925,24,4001,53,1.28,35,0
2,Kansas City Royals,2019,990,105,45,954,8421.0,6,2807,3842,...,0.346,543,5,816,24,4125,39,1.46,34,0
3,Houston Astros,2019,875,54,50,954,8589.0,6,2863,3788,...,0.284,432,7,1074,27,3929,67,1.14,31,0
4,Tampa Bay Rays,2019,975,92,53,963,8760.0,11,2920,3948,...,0.291,409,6,1037,26,3985,59,1.16,40,0


In [41]:
# remove data from 2016 through 2019.
team_data_new = team_data.loc[team_data["year"] < 2016]
team_data_new.head()

,team,year,A,DP,E,GS2,INN,PB,PO,TC,...,OBP1,R1,SHO,SO1,SV,TBF,W,WHIP,WP,winners
120,San Francisco Giants,2015,1639,136,72,1458,13143.0,6,4381,6092,...,0.303,631,11,1309,43,6048,87,1.21,40,0
121,Washington Nationals,2015,1425,142,73,1458,13137.0,17,4379,5877,...,0.300,612,12,1476,46,6036,95,1.19,47,0
122,Houston Astros,2015,1599,135,77,1458,13212.0,18,4404,6080,...,0.314,701,8,1396,44,6180,84,1.29,98,0
123,Detroit Tigers,2015,1537,148,75,1449,12852.0,5,4284,5896,...,0.320,721,8,1232,47,6048,86,1.32,44,0
124,Boston Red Sox,2015,1427,139,75,1458,12957.0,37,4319,5821,...,0.314,694,5,1362,43,6073,93,1.27,52,0


In [42]:
target = team_data_new["winners"]
features = team_data_new.drop({"team", "year", "winners"}, axis=1)
feature_columns = list(features.columns)
print (target.shape)
print (features.shape)
print (feature_columns)

(2344,)
(2344, 46)
['A', 'DP', 'E', 'GS2', 'INN', 'PB', 'PO', 'TC', '2B', '3B', 'AB', 'AO', 'BB', 'H', 'HBP', 'HR', 'NP_x', 'OBP', 'OPS_x', 'R', 'RBI', 'SAC', 'SB', 'SLG', 'BB1', 'BK', 'CG', 'ER', 'ERA', 'G1', 'GF', 'GS', 'H1', 'HB', 'HR1', 'IP', 'L', 'OBP1', 'R1', 'SHO', 'SO1', 'SV', 'TBF', 'W', 'WHIP', 'WP']


#### STEP2: Upsample and scale data.

In [4]:
# reset the index.
team_data_new = team_data_new.reset_index().drop({"index"}, axis=1)
team_data_new.head()

,team,year,A,DP,E,GS2,INN,PB,PO,TC,...,OBP1,R1,SHO,SO1,SV,TBF,W,WHIP,WP,winners
0,San Francisco Giants,2015,1639,136,72,1458,13143.0,6,4381,6092,...,0.303,631,11,1309,43,6048,87,1.21,40,0
1,Washington Nationals,2015,1425,142,73,1458,13137.0,17,4379,5877,...,0.300,612,12,1476,46,6036,95,1.19,47,0
2,Houston Astros,2015,1599,135,77,1458,13212.0,18,4404,6080,...,0.314,701,8,1396,44,6180,84,1.29,98,0
3,Detroit Tigers,2015,1537,148,75,1449,12852.0,5,4284,5896,...,0.320,721,8,1232,47,6048,86,1.32,44,0
4,Boston Red Sox,2015,1427,139,75,1458,12957.0,37,4319,5821,...,0.314,694,5,1362,43,6073,93,1.27,52,0


In [43]:
# remove team and year.
feature_columns_new = feature_columns + ["winners"]
team_data_new = team_data[feature_columns_new]
team_data_new.head()

,A,DP,E,GS2,INN,PB,PO,TC,2B,3B,...,OBP1,R1,SHO,SO1,SV,TBF,W,WHIP,WP,winners
0,1033,114,43,936,8313.0,3,2771,3847,157,10,...,0.319,456,4,895,33,3896,56,1.29,21,0
1,1010,83,45,945,8538.0,2,2846,3901,203,27,...,0.315,472,7,925,24,4001,53,1.28,35,0
2,990,105,45,954,8421.0,6,2807,3842,185,31,...,0.346,543,5,816,24,4125,39,1.46,34,0
3,875,54,50,954,8589.0,6,2863,3788,200,15,...,0.284,432,7,1074,27,3929,67,1.14,31,0
4,975,92,53,963,8760.0,11,2920,3948,195,21,...,0.291,409,6,1037,26,3985,59,1.16,40,0


In [44]:
# upsample for a more balanced dataset.
def upsample(dataset, no_samples):
    '''
    INPUT: 
    -dataset = dataset without team names and year.
    -n_samples = number of minority_unsampled.
    
    OUTPUT:
    -X_train_scaled = scaled X train data.
    -X_test_scaled = scaled X test data.
    -y_train = y train data
    -y_test = y test data
    
    DESCRIPTION:
    -dataset is taken in and split into minority and majority classes.
    -dataset is then upsampled for the mainority class
    -split the data into features and targets
    -split data into train and test sets
    -train and test sets were are scaled.
    '''
    
    # separate majority and minority classes.
    df_majority = dataset.loc[dataset["winners"] == 0]
    df_minority = dataset.loc[dataset["winners"] == 1]

    # upsample minority class.
    df_minority_unsampled = resample(df_minority,
                                    replace=True,
                                    n_samples=no_samples,
                                    random_state=123)

    # combine majority class with upsampled minority class.
    df_upsampled = pd.concat([df_majority, df_minority_unsampled])

    # separate features and target.
    y = df_upsampled["winners"]
    X = df_upsampled[feature_columns]
    
    # split into train and test sets.
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

    # scale X_train and X_test.
    scaler = StandardScaler()

    # transform the training and testing data.
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.fit_transform(X_test)
    
    return X_train_scaled, X_test_scaled, y_train, y_test

In [45]:
# Do three different upsamplings.
X_train_100, X_test_100, y_train_100, y_test_100 = upsample(team_data_new, 2234)
X_train_50, X_test_50, y_train_50, y_test_50 = upsample(team_data_new, 1117)
X_train_25, X_test_25, y_train_25, y_test_25 = upsample(team_data_new, 559)

/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/Applications/anacon

#### STEP3: Grid Search Model--Logistic Regression.

In [46]:
def grid_search_logistic(X_train, X_test, y_train, y_test):
    '''
    INPUT: 
    -X_train = scaled X train data.
    -X_test = scaled X test data.
    -y_train = y train data.
    -y_test = y test data.
    
    OUTPUT:
    -classification report (has F1 score, precision and recall).
    -grid = saved model for prediction. 
    
    DESCRIPTION:
    -the scaled and split data is put through a grid search with logistic.
    -the model is trained.
    -a prediction is made.
    -print out the classification report and give the model.
    '''
    
    # fit the model.
    model = LogisticRegression(solver="lbfgs", max_iter= 2000)
    
    # create gridsearch estimator.
    param_grid = {"C": [0.001, 0.01, 0.1, 1, 10, 100]}
    grid = GridSearchCV(model, param_grid, verbose=3)

    # fit the model.
    grid.fit(X_train, y_train)

    # predict.
    prediction = grid.predict(X_test)
    
    # print out the basic information about the grid search.
    print (grid.best_params_)
    print (grid.best_score_)
    print (grid.best_estimator_)
    
    grid = grid.best_estimator_
    predictions = grid.predict(X_test)
    print (classification_report(y_test, prediction, target_names=["0", "1"]))
    
    return grid

In [47]:
# for 1 part 0 to 1 part 1
model_100 = grid_search_logistic(X_train_100, X_test_100, y_train_100, y_test_100)

/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


Fitting 3 folds for each of 6 candidates, totalling 18 fits
[CV] C=0.001 .........................................................
[CV] ................ C=0.001, score=0.7096774193548387, total=   0.0s
[CV] C=0.001 .........................................................
[CV] ................ C=0.001, score=0.7329842931937173, total=   0.0s
[CV] C=0.001 .........................................................
[CV] ................ C=0.001, score=0.7161572052401747, total=   0.0s
[CV] C=0.01 ..........................................................
[CV] ................. C=0.01, score=0.7253705318221447, total=   0.0s
[CV] C=0.01 ..........................................................
[CV] ................. C=0.01, score=0.7382198952879581, total=   0.0s
[CV] C=0.01 ..........................................................
[CV] ................. C=0.01, score=0.7222707423580786, total=   0.0s
[CV] C=0.1 ...........................................................
[CV] ............

[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed:    1.6s finished


{'C': 100}
0.7646887725421757
LogisticRegression(C=100, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=2000, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)
              precision    recall  f1-score   support

           0       0.78      0.72      0.75       607
           1       0.71      0.77      0.74       540

   micro avg       0.74      0.74      0.74      1147
   macro avg       0.74      0.74      0.74      1147
weighted avg       0.75      0.74      0.74      1147



In [48]:
# for 1 part 0 to 0.5 part 1
model_50 = grid_search_logistic(X_train_50, X_test_50, y_train_50, y_test_50)

/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


Fitting 3 folds for each of 6 candidates, totalling 18 fits
[CV] C=0.001 .........................................................
[CV] ................ C=0.001, score=0.7350230414746544, total=   0.0s
[CV] C=0.001 .........................................................
[CV] ................ C=0.001, score=0.7220299884659747, total=   0.0s
[CV] C=0.001 .........................................................
[CV] ................ C=0.001, score=0.7066974595842956, total=   0.0s
[CV] C=0.01 ..........................................................
[CV] ................. C=0.01, score=0.7419354838709677, total=   0.0s
[CV] C=0.01 ..........................................................
[CV] ................. C=0.01, score=0.7531718569780853, total=   0.0s
[CV] C=0.01 ..........................................................
[CV] ................. C=0.01, score=0.7159353348729792, total=   0.0s
[CV] C=0.1 ...........................................................
[CV] ............

[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed:    1.2s finished


In [49]:
# for 1 part 0 to 0.25 part 1
model_25 = grid_search_logistic(X_train_25, X_test_25, y_train_25, y_test_25)

/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


Fitting 3 folds for each of 6 candidates, totalling 18 fits
[CV] C=0.001 .........................................................
[CV] ................ C=0.001, score=0.8118131868131868, total=   0.0s
[CV] C=0.001 .........................................................
[CV] ................ C=0.001, score=0.8118131868131868, total=   0.0s
[CV] C=0.001 .........................................................
[CV] ................ C=0.001, score=0.8112947658402204, total=   0.0s
[CV] C=0.01 ..........................................................
[CV] ................. C=0.01, score=0.8159340659340659, total=   0.0s
[CV] C=0.01 ..........................................................
[CV] ................. C=0.01, score=0.8255494505494505, total=   0.0s
[CV] C=0.01 ..........................................................
[CV] ................. C=0.01, score=0.8305785123966942, total=   0.0s
[CV] C=0.1 ...........................................................
[CV] ............

[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed:    0.9s finished


This is not significantly better than the straight logistic regression.

#### STEP4: Grid Search Model--SVC.

In [50]:
def grid_search_svc(X_train, X_test, y_train, y_test):
    '''
    INPUT: 
    -X_train = scaled X train data.
    -X_test = scaled X test data.
    -y_train = y train data.
    -y_test = y test data.
    
    OUTPUT:
    -classification report (has F1 score, precision and recall).
    -grid = saved model for prediction. 
    
    DESCRIPTION:
    -the scaled and split data is put through a grid search with svc.
    -the model is trained.
    -a prediction is made.
    -print out the classification report and give the model.
    '''
    
    # set up svc model.
    model = SVC(kernel="rbf", probability=True)

    # create gridsearch estimator.
    param_grid = {"C": [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
                 "gamma": [0.0001, 0.001, 0.01, 0.1]}
    grid = GridSearchCV(model, param_grid, verbose=3)

    # fit the model.
    grid.fit(X_train, y_train)

    # predict.
    prediction = grid.predict(X_test)
    
    # print out the basic information about the grid search.
    print (grid.best_params_)
    print (grid.best_score_)
    print (grid.best_estimator_)
    
    grid = grid.best_estimator_
    predictions = grid.predict(X_test)
    print (classification_report(y_test, prediction, target_names=["0", "1"]))
    
    return grid

In [51]:
# for 1 part 0 to 1 part 1
model_100 = grid_search_svc(X_train_100, X_test_100, y_train_100, y_test_100)

/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 28 candidates, totalling 84 fits
[CV] C=0.0001, gamma=0.0001 ..........................................
[CV] . C=0.0001, gamma=0.0001, score=0.5074106364428945, total=   1.8s
[CV] C=0.0001, gamma=0.0001 ..........................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.1s remaining:    0.0s


[CV] . C=0.0001, gamma=0.0001, score=0.5069808027923212, total=   1.7s
[CV] C=0.0001, gamma=0.0001 ..........................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    4.0s remaining:    0.0s


[CV] . C=0.0001, gamma=0.0001, score=0.5074235807860262, total=   1.7s
[CV] C=0.0001, gamma=0.001 ...........................................
[CV] .. C=0.0001, gamma=0.001, score=0.5074106364428945, total=   1.9s
[CV] C=0.0001, gamma=0.001 ...........................................
[CV] .. C=0.0001, gamma=0.001, score=0.5069808027923212, total=   1.9s
[CV] C=0.0001, gamma=0.001 ...........................................
[CV] .. C=0.0001, gamma=0.001, score=0.5074235807860262, total=   2.0s
[CV] C=0.0001, gamma=0.01 ............................................
[CV] ... C=0.0001, gamma=0.01, score=0.5074106364428945, total=   1.8s
[CV] C=0.0001, gamma=0.01 ............................................
[CV] ... C=0.0001, gamma=0.01, score=0.5069808027923212, total=   1.8s
[CV] C=0.0001, gamma=0.01 ............................................
[CV] ... C=0.0001, gamma=0.01, score=0.5074235807860262, total=   1.9s
[CV] C=0.0001, gamma=0.1 .............................................
[CV] .

[CV] ..... C=10, gamma=0.0001, score=0.7384481255448997, total=   1.4s
[CV] C=10, gamma=0.0001 ..............................................
[CV] ..... C=10, gamma=0.0001, score=0.7399650959860384, total=   1.4s
[CV] C=10, gamma=0.0001 ..............................................
[CV] ..... C=10, gamma=0.0001, score=0.7100436681222707, total=   1.4s
[CV] C=10, gamma=0.001 ...............................................
[CV] ...... C=10, gamma=0.001, score=0.7602441150828247, total=   1.3s
[CV] C=10, gamma=0.001 ...............................................
[CV] ...... C=10, gamma=0.001, score=0.7914485165794066, total=   1.3s
[CV] C=10, gamma=0.001 ...............................................
[CV] ...... C=10, gamma=0.001, score=0.7606986899563318, total=   1.2s
[CV] C=10, gamma=0.01 ................................................
[CV] ....... C=10, gamma=0.01, score=0.9032258064516129, total=   0.9s
[CV] C=10, gamma=0.01 ................................................
[CV] .

[Parallel(n_jobs=1)]: Done  84 out of  84 | elapsed:  2.4min finished


{'C': 10, 'gamma': 0.1}
0.9880744618964514
SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.1, kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
              precision    recall  f1-score   support

           0       1.00      0.99      0.99       607
           1       0.99      1.00      0.99       540

   micro avg       0.99      0.99      0.99      1147
   macro avg       0.99      0.99      0.99      1147
weighted avg       0.99      0.99      0.99      1147



In [52]:
# for 1 part 0 to 0.5 part 1
model_50 = grid_search_svc(X_train_50, X_test_50, y_train_50, y_test_50)

/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 28 candidates, totalling 84 fits
[CV] C=0.0001, gamma=0.0001 ..........................................
[CV] . C=0.0001, gamma=0.0001, score=0.6705069124423964, total=   0.7s
[CV] C=0.0001, gamma=0.0001 ..........................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.8s remaining:    0.0s


[CV] .. C=0.0001, gamma=0.0001, score=0.671280276816609, total=   0.7s
[CV] C=0.0001, gamma=0.0001 ..........................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.6s remaining:    0.0s


[CV] . C=0.0001, gamma=0.0001, score=0.6709006928406467, total=   0.7s
[CV] C=0.0001, gamma=0.001 ...........................................
[CV] .. C=0.0001, gamma=0.001, score=0.6705069124423964, total=   0.7s
[CV] C=0.0001, gamma=0.001 ...........................................
[CV] ... C=0.0001, gamma=0.001, score=0.671280276816609, total=   0.7s
[CV] C=0.0001, gamma=0.001 ...........................................
[CV] .. C=0.0001, gamma=0.001, score=0.6709006928406467, total=   0.7s
[CV] C=0.0001, gamma=0.01 ............................................
[CV] ... C=0.0001, gamma=0.01, score=0.6705069124423964, total=   0.7s
[CV] C=0.0001, gamma=0.01 ............................................
[CV] .... C=0.0001, gamma=0.01, score=0.671280276816609, total=   0.7s
[CV] C=0.0001, gamma=0.01 ............................................
[CV] ... C=0.0001, gamma=0.01, score=0.6709006928406467, total=   0.7s
[CV] C=0.0001, gamma=0.1 .............................................
[CV] .

[CV] ................... C=10, gamma=0.0001, score=0.75, total=   0.8s
[CV] C=10, gamma=0.0001 ..............................................
[CV] ..... C=10, gamma=0.0001, score=0.7381776239907728, total=   0.8s
[CV] C=10, gamma=0.0001 ..............................................
[CV] ..... C=10, gamma=0.0001, score=0.7228637413394919, total=   0.8s
[CV] C=10, gamma=0.001 ...............................................
[CV] ...... C=10, gamma=0.001, score=0.7661290322580645, total=   0.7s
[CV] C=10, gamma=0.001 ...............................................
[CV] ...... C=10, gamma=0.001, score=0.7773933102652826, total=   0.7s
[CV] C=10, gamma=0.001 ...............................................
[CV] ...... C=10, gamma=0.001, score=0.7355658198614319, total=   0.7s
[CV] C=10, gamma=0.01 ................................................
[CV] ....... C=10, gamma=0.01, score=0.8525345622119815, total=   0.6s
[CV] C=10, gamma=0.01 ................................................
[CV] .

[Parallel(n_jobs=1)]: Done  84 out of  84 | elapsed:  1.1min finished


{'C': 10, 'gamma': 0.1}
0.9846212995001923
SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.1, kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
              precision    recall  f1-score   support

           0       1.00      0.99      0.99       606
           1       0.97      1.00      0.98       261

   micro avg       0.99      0.99      0.99       867
   macro avg       0.98      0.99      0.99       867
weighted avg       0.99      0.99      0.99       867



In [53]:
# for 1 part 0 to 0.25 part 1
model_25 = grid_search_svc(X_train_25, X_test_25, y_train_25, y_test_25)

/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 28 candidates, totalling 84 fits
[CV] C=0.0001, gamma=0.0001 ..........................................
[CV] . C=0.0001, gamma=0.0001, score=0.8118131868131868, total=   0.3s
[CV] C=0.0001, gamma=0.0001 ..........................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s


[CV] . C=0.0001, gamma=0.0001, score=0.8118131868131868, total=   0.3s
[CV] C=0.0001, gamma=0.0001 ..........................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.7s remaining:    0.0s


[CV] . C=0.0001, gamma=0.0001, score=0.8126721763085399, total=   0.3s
[CV] C=0.0001, gamma=0.001 ...........................................
[CV] .. C=0.0001, gamma=0.001, score=0.8118131868131868, total=   0.3s
[CV] C=0.0001, gamma=0.001 ...........................................
[CV] .. C=0.0001, gamma=0.001, score=0.8118131868131868, total=   0.3s
[CV] C=0.0001, gamma=0.001 ...........................................
[CV] .. C=0.0001, gamma=0.001, score=0.8126721763085399, total=   0.3s
[CV] C=0.0001, gamma=0.01 ............................................
[CV] ... C=0.0001, gamma=0.01, score=0.8118131868131868, total=   0.3s
[CV] C=0.0001, gamma=0.01 ............................................
[CV] ... C=0.0001, gamma=0.01, score=0.8118131868131868, total=   0.3s
[CV] C=0.0001, gamma=0.01 ............................................
[CV] ... C=0.0001, gamma=0.01, score=0.8126721763085399, total=   0.3s
[CV] C=0.0001, gamma=0.1 .............................................
[CV] .

[CV] ..... C=10, gamma=0.0001, score=0.8118131868131868, total=   0.4s
[CV] C=10, gamma=0.0001 ..............................................
[CV] ..... C=10, gamma=0.0001, score=0.8118131868131868, total=   0.4s
[CV] C=10, gamma=0.0001 ..............................................
[CV] ..... C=10, gamma=0.0001, score=0.8126721763085399, total=   0.4s
[CV] C=10, gamma=0.001 ...............................................
[CV] ...... C=10, gamma=0.001, score=0.8118131868131868, total=   0.3s
[CV] C=10, gamma=0.001 ...............................................
[CV] ...... C=10, gamma=0.001, score=0.8118131868131868, total=   0.3s
[CV] C=10, gamma=0.001 ...............................................
[CV] ...... C=10, gamma=0.001, score=0.8126721763085399, total=   0.3s
[CV] C=10, gamma=0.01 ................................................
[CV] ....... C=10, gamma=0.01, score=0.8763736263736264, total=   0.3s
[CV] C=10, gamma=0.01 ................................................
[CV] .

[Parallel(n_jobs=1)]: Done  84 out of  84 | elapsed:   32.8s finished


{'C': 10, 'gamma': 0.1}
0.965627864344638
SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.1, kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
              precision    recall  f1-score   support

           0       0.98      0.99      0.98       579
           1       0.94      0.91      0.93       149

   micro avg       0.97      0.97      0.97       728
   macro avg       0.96      0.95      0.96       728
weighted avg       0.97      0.97      0.97       728



Huh.  That's pretty good.  Try out model_100 and model_50 with the 2016-2018 stuff.

#### STEP5: Predict 2016-2018 winners with SVC Grid Search.

In [56]:
def predict_the_winner(model, year, team_data, X_train):
    '''
    INPUT: 
    -X_train = scaled X train data.
    -model = the saved model.
    -team_data = complete dataframe with all data.
    -year = the year want to look at.
    
    OUTPUT:
    -printed prediction.
    
    DESCRIPTION:
    -data from year of interest is isolated.
    -the data are scaled.
    -the prediction is made.
    -print out the resulting probability and the name of the team.
    '''
    
    # grab the data.
    team_data = team_data.loc[team_data["year"] == year].reset_index()

    # set features (no team, year, winners).
    # set target (winners).
    features = team_data[feature_columns]
    
    # scale X_train and X_test.
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    features = scaler.fit_transform(features)
    
    # fit the model.
    probabilities = model.predict_proba(features)

    # convert predictions to datafram.e
    WS_predictions = pd.DataFrame(probabilities[:,1])

    # Sort the DataFrame (descending)
    WS_predictions = WS_predictions.sort_values(0, ascending=False)

    WS_predictions['Probability'] = WS_predictions[0]

    # Print 50 highest probability HoF inductees from still eligible players
    for i, row in WS_predictions.head(50).iterrows():
       prob = ' '.join(('WS Probability =', str(row['Probability'])))
       print('')
       print(prob)
       print(team_data.iloc[i,1:27]["team"])

In [57]:
# predict for 2018.
predict_the_winner(model_100, 2018, team_data, X_train_100)


WS Probability = 8.864107483724929e-07
Arizona Diamondbacks

WS Probability = 5.987098028070572e-07
St. Louis Cardinals

WS Probability = 5.98163813121816e-07
Boston Red Sox

WS Probability = 4.4815220068323746e-07
Oakland Athletics

WS Probability = 3.962716981674135e-07
Chicago Cubs

WS Probability = 3.411498146515525e-07
Texas Rangers

WS Probability = 3.0611254464763705e-07
Tampa Bay Rays

WS Probability = 3.008803113780851e-07
Cleveland Indians

WS Probability = 2.9969412729194816e-07
Washington Nationals

WS Probability = 2.624570110629863e-07
Miami Marlins

WS Probability = 2.5656831591710523e-07
Houston Astros

WS Probability = 2.4931680185652157e-07
Chicago White Sox

WS Probability = 2.3908142219865127e-07
Detroit Tigers

WS Probability = 2.365589990608981e-07
Minnesota Twins

WS Probability = 2.3424856590524415e-07
Los Angeles Dodgers

WS Probability = 2.3105234057688985e-07
New York Yankees

WS Probability = 2.1378597732862957e-07
Cincinnati Reds

WS Probability = 2.017457

/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [58]:
# predict for 2017.
predict_the_winner(model_100, 2017, team_data, X_train_100)


WS Probability = 6.309547022395061e-07
Los Angeles Angels

WS Probability = 4.1316548755350435e-07
Los Angeles Dodgers

WS Probability = 3.835897084776093e-07
Washington Nationals

WS Probability = 3.624742824150671e-07
Houston Astros

WS Probability = 3.101140353653143e-07
Tampa Bay Rays

WS Probability = 2.986143656985992e-07
Milwaukee Brewers

WS Probability = 2.913240946741632e-07
Kansas City Royals

WS Probability = 2.654249317623736e-07
Oakland Athletics

WS Probability = 2.556190487656305e-07
Chicago Cubs

WS Probability = 2.526465321318717e-07
Chicago White Sox

WS Probability = 2.5114609504522737e-07
Boston Red Sox

WS Probability = 2.464308863625614e-07
Cleveland Indians

WS Probability = 2.1785713349950006e-07
Baltimore Orioles

WS Probability = 2.068264628787461e-07
Miami Marlins

WS Probability = 1.9506230005874968e-07
Atlanta Braves

WS Probability = 1.8584064021875782e-07
Arizona Diamondbacks

WS Probability = 1.8288574080318613e-07
Colorado Rockies

WS Probability = 1.

/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [59]:
# predict for 2016
predict_the_winner(model_100, 2016, team_data, X_train_100)


WS Probability = 4.153423869081854e-07
Oakland Athletics

WS Probability = 3.418554841012313e-07
Colorado Rockies

WS Probability = 3.201934524689137e-07
Minnesota Twins

WS Probability = 2.829773524682836e-07
New York Yankees

WS Probability = 2.7442657594473827e-07
Seattle Mariners

WS Probability = 2.664900795216095e-07
Washington Nationals

WS Probability = 2.6520664636780334e-07
Houston Astros

WS Probability = 2.483598390930739e-07
Cleveland Indians

WS Probability = 2.4634124158230823e-07
Boston Red Sox

WS Probability = 2.3371032242560324e-07
Los Angeles Dodgers

WS Probability = 2.166651666781273e-07
Chicago Cubs

WS Probability = 2.0442837094447964e-07
San Diego Padres

WS Probability = 1.9937877061035925e-07
San Francisco Giants

WS Probability = 1.9734661635863432e-07
Detroit Tigers

WS Probability = 1.7952147727077784e-07
Los Angeles Angels

WS Probability = 1.70643309958154e-07
St. Louis Cardinals

WS Probability = 1.547942227535032e-07
Chicago White Sox

WS Probability 

/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [60]:
# predict for 2018
predict_the_winner(model_50, 2018, team_data, X_train_50)


WS Probability = 8.253702141744944e-07
Boston Red Sox

WS Probability = 7.693392989541881e-07
Arizona Diamondbacks

WS Probability = 6.468289844282572e-07
Cleveland Indians

WS Probability = 5.93288662914052e-07
St. Louis Cardinals

WS Probability = 5.707352450627377e-07
Texas Rangers

WS Probability = 5.35517687313307e-07
Tampa Bay Rays

WS Probability = 5.250018943602652e-07
Washington Nationals

WS Probability = 4.24496547476341e-07
Oakland Athletics

WS Probability = 4.164508977807319e-07
Chicago Cubs

WS Probability = 4.157356940666007e-07
Chicago White Sox

WS Probability = 4.138881415327555e-07
Miami Marlins

WS Probability = 4.0293996255279086e-07
Detroit Tigers

WS Probability = 3.776601795367539e-07
Los Angeles Dodgers

WS Probability = 3.566284993503818e-07
Houston Astros

WS Probability = 3.503497811403545e-07
Seattle Mariners

WS Probability = 3.5003033911028414e-07
Cincinnati Reds

WS Probability = 3.175445340269904e-07
Minnesota Twins

WS Probability = 3.162011254572118

/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [61]:
# predict for 2017
predict_the_winner(model_50, 2017, team_data, X_train_50)


WS Probability = 1.6951262470921632e-06
Los Angeles Angels

WS Probability = 1.1373225307926398e-06
Washington Nationals

WS Probability = 7.391083132680538e-07
Tampa Bay Rays

WS Probability = 7.016864740140463e-07
Milwaukee Brewers

WS Probability = 6.537171144600581e-07
Houston Astros

WS Probability = 6.503790512775786e-07
Los Angeles Dodgers

WS Probability = 4.882447273320116e-07
Chicago White Sox

WS Probability = 4.6655917636626726e-07
Kansas City Royals

WS Probability = 4.3633842870951563e-07
Cleveland Indians

WS Probability = 4.141233453618528e-07
Chicago Cubs

WS Probability = 4.065079289210853e-07
Boston Red Sox

WS Probability = 4.0515130622713334e-07
Oakland Athletics

WS Probability = 3.8201644919374816e-07
Baltimore Orioles

WS Probability = 3.5595161820542015e-07
Miami Marlins

WS Probability = 3.543700126789747e-07
Atlanta Braves

WS Probability = 2.830490026391126e-07
Pittsburgh Pirates

WS Probability = 2.613220577811357e-07
New York Yankees

WS Probability = 2.5

/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [62]:
# predict for 2016
predict_the_winner(model_50, 2016, team_data, X_train_50)


WS Probability = 1.2949854768342833e-06
Oakland Athletics

WS Probability = 7.353647384048838e-07
Seattle Mariners

WS Probability = 6.782691470404411e-07
Washington Nationals

WS Probability = 4.6579460935426256e-07
New York Yankees

WS Probability = 4.474934273901575e-07
Minnesota Twins

WS Probability = 4.270847484468139e-07
Cleveland Indians

WS Probability = 4.1371670930013236e-07
Boston Red Sox

WS Probability = 4.044361070810831e-07
Houston Astros

WS Probability = 3.926133625335706e-07
Los Angeles Dodgers

WS Probability = 3.640054174137475e-07
San Diego Padres

WS Probability = 3.336376654534898e-07
Detroit Tigers

WS Probability = 3.181680594327744e-07
Colorado Rockies

WS Probability = 3.17747033395235e-07
San Francisco Giants

WS Probability = 2.8465421262716477e-07
Chicago White Sox

WS Probability = 2.3269658485852134e-07
Cincinnati Reds

WS Probability = 2.2215601242318686e-07
Baltimore Orioles

WS Probability = 1.6685541590252813e-07
Philadelphia Phillies

WS Probabili

/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


This is not better.  The logistic regression with grid and upsampling works just as good as the straight up logistic regression.  SVC didn't work well. Next!  This is overfitting!

## Grid search with no upsampling, with scaling.

In [67]:
# upsample for a more balanced dataset.
def splited(dataset):
    '''
    INPUT: 
    -dataset = dataset without team names and year.
    
    OUTPUT:
    -X_train_scaled = scaled X train data.
    -X_test_scaled = scaled X test data.
    -y_train = y train data
    -y_test = y test data
    
    DESCRIPTION:
    -split the data into features and targets
    -split data into train and test sets
    -train and test sets were are scaled.
    '''
    
    # separate features and target.
    y = dataset["winners"]
    X = dataset[feature_columns]
    
    # split into train and test sets.
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

    # scale X_train and X_test.
    scaler = StandardScaler()

    # transform the training and testing data.
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.fit_transform(X_test)
    
    return X_train_scaled, X_test_scaled, y_train, y_test

In [68]:
X_train, X_test, y_train, y_test = splited(team_data_new)

/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [ ]:
modgrid_search_svc

In [ ]:
def grid_search_svc(X_train, X_test, y_train, y_test):
    '''
    INPUT: 
    -X_train = scaled X train data.
    -X_test = scaled X test data.
    -y_train = y train data.
    -y_test = y test data.
    
    OUTPUT:
    -classification report (has F1 score, precision and recall).
    -grid = saved model for prediction. 
    
    DESCRIPTION:
    -the scaled and split data is put through a grid search with svc.
    -the model is trained.
    -a prediction is made.
    -print out the classification report and give the model.
    '''
    
    # set up svc model.
    model = SVC(kernel="rbf", probability=True)

    # create gridsearch estimator.
    param_grid = {"C": [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
                 "gamma": [0.0001, 0.001, 0.01, 0.1]}
    grid = GridSearchCV(model, param_grid, verbose=3)

    # fit the model.
    grid.fit(X_train, y_train)

    # predict.
    prediction = grid.predict(X_test)
    
    # print out the basic information about the grid search.
    print (grid.best_params_)
    print (grid.best_score_)
    print (grid.best_estimator_)
    
    grid = grid.best_estimator_
    predictions = grid.predict(X_test)
    print (classification_report(y_test, prediction, target_names=["0", "1"]))
    
    return grid